<center> <h1>libOmexMeta Supplementary Content </h1> </center>
This document describes how to use libOmexMeta version 1.1. We split this document
into two sections. In part A we provide a tutorial in which we focus on providing use case examples
for each type of annotation supported with a narrative to explain the annotations. Part B is example
focused and demonstrates for programmers how one is likely to use pyomexmeta, the Python front end
of libOmexMeta. Please note that throughout this document we use examples that are intended to be illustrative
with regards to annotation, but not biologically accurate or entirely realistical modelling scenarios.

We further wish to emphasize that the users of OMEX metadata libraries are programmers,
rather than biologists. As described in the manuscript, if tool developers use libOmexMeta or
pyomexmeta libraries, then their tools, which are used by biologists, will produce annotations on models
that are compliant with the standard, and therefore will better support findability,
accessibility, interoperability, and reusability of those models.

<center> <h2>Part A: libOmexMeta Tutorial</h2></center>

<left> <h3> 1. Model-level annotations </h3> </left>
We begin by demonstrating how to create annotations to indicate the creator,
curator, taxon, publication ID, and date created for an sbml model. The SBML model that we use in this example is nothing
more than a basic toy model where A transitions to B using first order mass action kinetics.

In [2]:
from pyomexmeta import RDF, eUriType

sbml = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="ToyModel" id="ToyModel">
    <listOfCompartments>
      <compartment sboTerm="SBO:0000410" id="cell" metaid="cell" spatialDimensions="3" size="1" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="A" metaid="A" compartment="cell" initialConcentration="10" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="B" metaid="B" compartment="cell" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="k1" value="0.1" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="A" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="B" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> k1 </ci>
              <ci> A </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


rdf = RDF()
rdf.set_archive_uri("MyOmexArchive.omex")
rdf.set_model_uri("MyModel.sbml")
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)
annot_editor.add_creator("orcid:0000-0001-8254-4957") \
    .add_curator("orcid:0000-0001-8254-4958") \
    .add_taxon("taxon/9895") \
    .add_pubmed("pubmed/12334") \
    .add_date_created("2020-12-16")

print(rdf)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix pubmed: <https://identifiers.org/pubmed:> .
@prefix bqmodel: <http://biomodels.net/model-qualifiers/> .
@prefix NCBI_Taxon: <https://identifiers.org/taxonomy:> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/MyOmexArchive.omex/> .
@prefix local: <http://omex-library.org/MyOmexArchive.omex/MyModel.rdf#> .

<http://omex-library.org/MyOmexArchive.omex/MyModel.rdf#>
    dc:creator <https://orcid.org/orcid:0000-0001-8254-4958> .

<http://omex-library.org/MyOmexArchive.omex/MyModel.sbml#>
    bqbiol:hasTaxon <https://identifiers.org/taxonomy:taxon/9895> ;
    bqmodel:isDescribedBy <https://identifiers.org/pubmed:pubmed/12334> ;
    dc:created [
        dc:W3CDTF "2020-09-18"^^rdf:string
    ] ;
    dc:creator <https://orcid.org/orcid:0000-0001-8254-4957> .




As described in the OmexMeta v1.1 specification, it is also optionally possible to specify
the people (curators, creators) with strings for names, email addresses, etc.

<left> <h3> 2. Singular Annotations </h3> </left>
In this section, we demonstrate how one can create a single annotation which is an RDF triple
object, comprising a subject, predicate and a object. We reuse the already defined `sbml` string
variable from section 1.

In [3]:
from pyomexmeta import RDF, eUriType
rdf = RDF()
rdf.set_archive_uri("MyOmexArchive.omex")
rdf.set_model_uri("MyModel.sbml")
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_singular_annotation() as singular_annotation:
    # CHEBI:16236 = EtOH in chebi
     singular_annotation.about("A") \
                        .predicate("bqbiol", "is") \
                        .resource_uri("CHEBI:16236")
print(rdf)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/MyOmexArchive.omex/> .
@prefix local: <http://omex-library.org/MyOmexArchive.omex/MyModel.rdf#> .

<http://omex-library.org/MyOmexArchive.omex/MyModel.sbml#A>
    bqbiol:is <https://identifiers.org/CHEBI:16236> .




This example shows the lowest level interface for creating an annotation. These three methods
demonstrate how to create the [subject, predicate and resource](https://sys-bio.github.io/libOmexMeta/docs-build/editing_rdf/editing_rdf.html#subjects-predicates-and-resources)
parts of an RDF triple object. This annotation states that the SBML species with metaid `A` is the
chemical entity that has the CHEBI identifier `CHEBI:16236`: ethanol.

<left> <h3> 3. Composite Annotations </h3> </left>
<left> <h4> 3. SBML </h4> </left>
Composite annotations are collections of singlular annotations with a predefined structure. Three types
of composite annotation are supported by libOmexMeta v1.1: annotation for a physical entity; a physical process
or a energy differential. In this section we describe these further.

<left> <h5> 3.1 Annotations for a Physical Entity </h5> </left>

In many modeling situations, one may wish to annotate both the physical entity and the physical property tha
is being simulated, such as concentration, particle numbers, fluid volume or fluid pressure.
This allows us to capture more complete biological semantics. For instance, in a cardiovascular fluid flow model, one can model
either the fluid volume or fluid pressure inside the left ventricle. In chemical
kinetic models, one can model the chemical concentration or the amount in absolute numbers of a physical
entity, such as glucose or ethanol.

In some modeling languages, only one of these aspects may be present in the code.
For example, in CellML, one has variables that represent chemical concentrations,
or blood pressures, but the biological physical entity is implicit. Conversely, in SBML,
the physical entity is specified as a “species”, but the physical property (usually chemical
concentration or particle numbers) is left implicit. Thus, for annotation, we must allow for either situation.

A physical entity may be a complex structured object. For example, one may need to distinguish
between niclear and cytoplasmic glucose. These are both the same entity (glucose), but
they are part of different anatomic structures. In the following example we demonstrate how
to annotate such a situation using another toy model depicting glucose in dynamic equilibrium
between two compartments, the nucleus and the cytoplasm.

In [ ]:
from pyomexmeta import RDF, eUriType

sbml = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="GlucoseTransport" id="GlucoseTransport">
    <listOfCompartments>
      <compartment id="cytoplasm" metaid="cytoplasm" spatialDimensions="3" size="1" constant="true"/>
      <compartment id="nucleus" metaid="nucleus" spatialDimensions="3" size="1" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="glucose_c" metaid="glucose_c" compartment="cytoplasm" initialConcentration="10" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="glucose_n" metaid="glucose_n" compartment="nucleus" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kimp" metaid="kimp" value="10" constant="true"/>
      <parameter id="kexp" metaid="kexp" value="0.1" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="r1_imp"  metaid="r1_imp" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="glucose_c" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="glucose_n" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> cytoplasm </ci>
              <ci> kimp </ci>
              <ci> glucose_c </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="r2_exp" metaid="r2_exp" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="glucose_n" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="glucose_c" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> nucleus </ci>
              <ci> kexp </ci>
              <ci> glucose_n </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""

rdf = RDF()
rdf.set_archive_uri("GlucoseTransport.omex")
rdf.set_model_uri("glucose_transport.sbml")
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = The GO cellular component term for cytoplasm.
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                   .identity("CHEBI:17234")\
                   .is_part_of("GO:0005737")   

with annot_editor.new_physical_entity() as nuclear_glucose:
    # CHEBI:17234 = glucose
    # CHEBI:17234 = The GO cellular component term for nucleus.
    nuclear_glucose.about("glucose_n", eUriType.MODEL_URI) \
                   .identity("CHEBI:17234")\
                   .is_part_of("GO:0005634")   
    
print(rdf)

In this example we construct two physical entity annotations, one each for cytoplasmic and nuclear glucose. The
argument to the `about` method is the metaid for the SBML species element called `glucose_c`.

      <species id="glucose_c" metaid="glucose_c" compartment="cytoplasm" initialConcentration="10" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>

Internally, libOmexMeta creates a new metaid (`local:EntityProperty0001`) which is `local` to the annotation document we are creating that
links the species with both `id` and metaid `glucose_c` to the information that annotates this physical entity.

    local:EntityProperty0000
        bqbiol:is <https://identifiers.org/CHEBI:17234> ;
        bqbiol:isPartOf <https://identifiers.org/GO:0005737> .

Often, the `is_part_of` relation is only important when entities can exist in multiple
compartments and the modeler needs to distinguish between them.

The `has_property` relation may optionally be used to indicate that the
property we are simulating is the chemical concentration of the entity.

In [ ]:
from pyomexmeta import RDF, eUriType 
rdf = RDF()
rdf.set_archive_uri("GlucoseTransport.omex")
rdf.set_model_uri("glucose_transport.sbml")
# resuing the sbml variable from the previous section
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = The GO cellular component term for cytoplasm.
    # OPB:00340 = chemical concentration
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                     .identity("CHEBI:17234")\
                     .is_part_of("GO:0005737")\
                     .has_property("OPB:00340")
    
print(rdf)

Note that the difference between this and the last example is the presence of the additional annotation
with the `bqbiol:isVersionOf` predicate:

    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .


<left> <h5> 3.2 Annotations for a Physical Process </h5> </left>
Perhaps the most prevalent type of biosimulation model is one that simulates biochemical reactions.
Thus, we provide special attention to annotations for biological processes, such as reactions.
Unlike physical entities, processes rarely have unique names. However, some processes can
be categorized as a “version of” a Gene Ontology process term. Thus, in the main text, we used
a “version of” alcohol dehydrogenase. Below, we provide the Python code for an expanded version
of this same reaction. Note, we reuse the sbml model from an earlier example but include the sbml
string to make the example self contained.

In [ ]:
from pyomexmeta import RDF, eUriType

sbml = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
    <model metaid="ADHModel" id="ADHModel">
        <listOfCompartments>
            <compartment id="cytosol" metaid="cytosol" spatialDimensions="3" size="1" constant="true"/>
        </listOfCompartments>
        <listOfSpecies>
            <species id="NAD" metaid="NAD" compartment="cytosol" initialConcentration="10" hasOnlySubstanceUnits="false"
                     boundaryCondition="false" constant="false"/>
            <species id="EtOH" metaid="EtOH" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
            <species id="NADH" metaid="NADH" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
            <species id="Aldehyde" metaid="Aldehyde" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
            <species id="ADH" metaid="ADH" compartment="cytosol" initialConcentration="1"
                     hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
        </listOfSpecies>
        <listOfParameters>
            <parameter id="vmax" metaid="k" value="0.5" constant="true"/>
        </listOfParameters>
        <listOfReactions>
            <reaction id="ADH" metaid="ADH" reversible="false" fast="false">
                <listOfReactants>
                    <speciesReference species="EtOH" stoichiometry="1" constant="true"/>
                    <speciesReference species="NAD" stoichiometry="1" constant="true"/>
                </listOfReactants>
                <listOfProducts>
                    <speciesReference species="Aldehyde" stoichiometry="1" constant="true"/>
                    <speciesReference species="NAD" stoichiometry="1" constant="true"/>
                </listOfProducts>
                <kineticLaw>
                    <math xmlns="http://www.w3.org/1998/Math/MathML">
                        <apply>
                            <times/>
                            <ci>k</ci>
                            <ci>ADH</ci>
                            <ci>EtOH</ci>
                            <ci>NAD</ci>
                        </apply>
                    </math>
                </kineticLaw>
            </reaction>
        </listOfReactions>
    </model>
</sbml>
"""

rdf = RDF()
rdf.set_archive_uri("AHD.omex")
rdf.set_model_uri("AHDModel.sbml")
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_process() as phy_process:
    phy_process \
        .about("ADH") \
        .is_version_of("GO:0004022") \
        .add_source(physical_entity_reference="EtOH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_source(physical_entity_reference="NAD", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="Aldehyde", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="NADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_mediator(physical_entity_reference="ADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .has_property("OPB:OPB_00237")                       #Chemical flow rate

print(rdf)

In the above, the reaction has two reactants, two products, and one mediating enzyme.
Note that stoichiometry, where appropriate, can be specified via the multiplier fields.
Like with the physical entity type composite annotations, the `about` method
provides the link between the annotation document we are composing and the SBML source code. Similarly,
the `physical_entity_reference` arguments are metaids to elements within the SBML document.

The final optional clause `has_property` indicates that the reaction being simulated has the physical property
of “chemical flow rate”.

<left> <h5> 3.3 Annotations for a energy differential </h5> </left>

*** attention This example needs work. Please provide a model and fix the annotation apropriately ***

Finally, we provide an example for annotating the electrical potential across the cellular membrane,
specifically resulting from calcium ions. In addition to source and sink, this is annotated with the
OPB term for the Nernst, or reversal potential.

The next example demonstrates the annotation of a physical entity in a model that simulates
hemodynamics. We annotate a term as capturing the blood pressure (physical property) within the aorta (physical entity).

In [ ]:
from pyomexmeta import RDF, eUriType
sbml = we need a model for this example.

rdf = RDF()
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_entity() as blood_pressure_entity:
    # opb/OPB:00509 = fluid pressure
    # fma/fma_9670 = portion of blood
    # #lumen of Aorta = lumen of aorta
    blood_pressure_entity.about("Pa001") \
                        .has_property("opb/OPB:00509") \     
                        .identity("fma/fma_9670") \   
                        .is_part_of("fma/fma_13096")       
    
    

In [ ]:
# Omex_S001: Calcium Ions extracellular space
# EnergyD001: The metaID of the SBML parameter
# Omex_S003: Calcium ions in cytoplasm
#OPB/OPB_01581: Nernst reversal potential
with annot_editor.new_physical_energyDiff() as energy_diff:
    energy_diff \
     .about("EnergyD001") \
     .add_source(physical_entity_reference="Omex_S001", uri_type=eUriType.MODEL_URI) \
     .add_sink(physical_entity_reference="Omex_S003", uri_type=eUriType.MODEL_URI) \
     .add_has_property("OPB/OPB_01581")

<left> <h4> 3. CellML </h4> </left>
space for cellml examples

<left> <h5> 3.1 Annotations for a Physical Entity </h5> </left>
...

In [4]:
from pyomexmeta import RDF, eUriType

cellml = """"""

rdf = RDF()
rdf.set_archive_uri("name.omex")
rdf.set_model_uri("name.cellml")
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

CHANGE ME

with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = The GO cellular component term for cytoplasm.
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                   .identity("CHEBI:17234")\
                   .is_part_of("GO:0005737")

with annot_editor.new_physical_entity() as nuclear_glucose:
    # CHEBI:17234 = glucose
    # CHEBI:17234 = The GO cellular component term for nucleus.
    nuclear_glucose.about("glucose_n", eUriType.MODEL_URI) \
                   .identity("CHEBI:17234")\
                   .is_part_of("GO:0005634")

print(rdf)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/GlucoseTransport.omex/> .
@prefix local: <http://omex-library.org/GlucoseTransport.omex/glucose_transport.rdf#> .

local:Property0000
    bqbiol:isPropertyOf <http://omex-library.org/GlucoseTransport.omex/glucose_transport.sbml#glucose_c>, <http://omex-library.org/GlucoseTransport.omex/glucose_transport.sbml#glucose_n> .

<http://omex-library.org/GlucoseTransport.omex/glucose_transport.sbml#glucose_c>
    bqbiol:is <https://identifiers.org/CHEBI:17234> ;
    bqbiol:isPartOf <https://identifiers.org/GO:0005737> .

<http://omex-library.org/GlucoseTransport.omex/glucose_transport.sbml#glucose_n>
    bqbiol:is <https://identifiers.org/CHEBI:17234> ;
    bqbiol:isPartOf <https://identifiers.org/GO:0005634> .




UPDATE ME FOR CELLML

In this example we construct two physical entity annotations, one each for cytoplasmic and nuclear glucose. The
argument to the `about` method is the metaid for the SBML species element called `glucose_c`.

      <species id="glucose_c" metaid="glucose_c" compartment="cytoplasm" initialConcentration="10" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>

Internally, libOmexMeta creates a new metaid (`local:EntityProperty0001`) which is `local` to the annotation document we are creating that
links the species with both `id` and metaid `glucose_c` to the information that annotates this physical entity.

    local:EntityProperty0000
        bqbiol:is <https://identifiers.org/CHEBI:17234> ;
        bqbiol:isPartOf <https://identifiers.org/GO:0005737> .

Often, the `is_part_of` relation is only important when entities can exist in multiple
compartments and the modeler needs to distinguish between them.

The `has_property` relation may optionally be used to indicate that the
property we are simulating is the chemical concentration of the entity.

In [ ]:
UPDATE ME

from pyomexmeta import RDF, eUriType
rdf = RDF()
rdf.set_archive_uri("GlucoseTransport.omex")
rdf.set_model_uri("glucose_transport.sbml")
# resuing the sbml variable from the previous section
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = The GO cellular component term for cytoplasm.
    # OPB:00340 = chemical concentration
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                     .identity("CHEBI:17234")\
                     .is_part_of("GO:0005737")\
                     .has_property("OPB:00340")

print(rdf)

Note that the difference between this and the last example is the presence of the additional annotation
with the `bqbiol:isVersionOf` predicate:

    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .


<left> <h5> 3.2 Annotations for a Physical Process </h5> </left>
Perhaps the most prevalent type of biosimulation model is one that simulates biochemical reactions.
Thus, we provide special attention to annotations for biological processes, such as reactions.
Unlike physical entities, processes rarely have unique names. However, some processes can
be categorized as a “version of” a Gene Ontology process term. Thus, in the main text, we used
a “version of” alcohol dehydrogenase. Below, we provide the Python code for an expanded version
of this same reaction. Note, we reuse the sbml model from an earlier example but include the sbml
string to make the example self contained.

In [ ]:
from pyomexmeta import RDF, eUriType

cellml = """"""""

UPDATE ME

rdf = RDF()
rdf.set_archive_uri("AHD.omex")
rdf.set_model_uri("AHDModel.sbml")
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_process() as phy_process:
    phy_process \
        .about("ADH") \
        .is_version_of("GO:0004022") \
        .add_source(physical_entity_reference="EtOH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_source(physical_entity_reference="NAD", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="Aldehyde", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="NADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_mediator(physical_entity_reference="ADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .has_property("OPB:OPB_00237")                       #Chemical flow rate

print(rdf)

UPDATE ME

In the above, the reaction has two reactants, two products, and one mediating enzyme.
Note that stoichiometry, where appropriate, can be specified via the multiplier fields.
Like with the physical entity type composite annotations, the `about` method
provides the link between the annotation document we are composing and the SBML source code. Similarly,
the `physical_entity_reference` arguments are metaids to elements within the SBML document.

The final optional clause `has_property` indicates that the reaction being simulated has the physical property
of “chemical flow rate”.

<left> <h5> 3.3 Annotations for a energy differential </h5> </left>

*** attention This example needs work. Please provide a model and fix the annotation apropriately ***

Finally, we provide an example for annotating the electrical potential across the cellular membrane,
specifically resulting from calcium ions. In addition to source and sink, this is annotated with the
OPB term for the Nernst, or reversal potential.

The next example demonstrates the annotation of a physical entity in a model that simulates
hemodynamics. We annotate a term as capturing the blood pressure (physical property) within the aorta (physical entity).

In [ ]:
from pyomexmeta import RDF, eUriType
crllml = we need a model for this example.

rdf = RDF()
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_entity() as blood_pressure_entity:
    # opb/OPB:00509 = fluid pressure
    # fma/fma_9670 = portion of blood
    # #lumen of Aorta = lumen of aorta
    blood_pressure_entity.about("Pa001") \
                        .has_property("opb/OPB:00509") \
                        .identity("fma/fma_9670") \
                        .is_part_of("fma/fma_13096")



In [ ]:
rdf = RDF()
rdf.set_archive_uri("AHD.omex")
rdf.set_model_uri("AHDModel.sbml")

# Omex_S001: Calcium Ions extracellular space
# EnergyD001: The metaID of the SBML parameter
# Omex_S003: Calcium ions in cytoplasm
#OPB/OPB_01581: Nernst reversal potential
with annot_editor.new_physical_energyDiff() as energy_diff:
    energy_diff \
     .about("EnergyD001") \
     .add_source(physical_entity_reference="Omex_S001", uri_type=eUriType.MODEL_URI) \
     .add_sink(physical_entity_reference="Omex_S003", uri_type=eUriType.MODEL_URI) \
     .add_has_property("OPB/OPB_01581")


<center> <h2> Part B: libOmexMeta Examples </h2> </center>
While the preceding section is designed to be more tutorial based to
guide users around the libOmexMeta library, this section is designed to be
example led, with an emphasis on how one is likely to use libOmexMeta in conjuction with `libcombine`.

In these examples, we use a simple SBML model that simulates the traditional Michaelis-Menten schema.     

In [ ]:
import glob
import os
import libcombine
from pyomexmeta import RDF, eUriType


SBML_STRING = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="MichaelisMenten" id="MichaelisMenten">
    <listOfCompartments>
      <compartment id="Cell" metaid="Cell" spatialDimensions="3" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S" metaid="S" compartment="Cell" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="E" metaid="E" compartment="Cell" initialConcentration="0.5" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="ES" metaid="ES" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="P" metaid="P" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kf" metaid="kf" value="0.1" constant="true"/>
      <parameter id="kb" metaid="kb" value="0.01" constant="true"/>
      <parameter id="kcat" metaid="kcat" value="0.0001" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kf </ci>
              <ci> S </ci>
              <ci> E </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R2" metaid="R2" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kb </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R3" metaid="R3" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
          <speciesReference species="P" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kcat </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


##########################################################################################
#   Helper functions
#   -----------------
#


def create_combine_archive(sbml, name, annotation_string=None):
    """
    Create a combine archive using libcombine. Archive will contain
    a single sbml file with the Manifest that is autocreated by libcombine.
    If annotation_string is not None, an additional annotation file will be
    stored in the combine containing annotation_string.
    Args:
        sbml: the sbml string to be put into a file
        name: Name used for archive and file in archive
        annotation_string: None or string containing the annotation for the combine archive

    Returns:

    """
    current_directory = dir_path = os.path.dirname(os.path.realpath(__file__))
    combine_archive_filename = os.path.join(current_directory, f"{name}.omex")
    archive = libcombine.CombineArchive()
    archive.addFileFromString(
        content=sbml,  # the string contining sbml
        targetName=f"{name}.sbml",  # name the content in archive
        format="sbml",  # format of this content
        isMaster=True  # boolean indicating whether the file should be opened first if there are multiple ones.
    )
    if annotation_string:
        archive.addFileFromString(
            annotation_string,
            f"{name}.rdf",
            "turtle",
            False
        )
    archive.writeToFile(combine_archive_filename)
    if not os.path.isfile(combine_archive_filename):
        raise FileNotFoundError(combine_archive_filename)

    print(f"Archive written to \"{combine_archive_filename}\"")
    return combine_archive_filename


def extract_sbml_from_combine_archive(archive_path):
    """
    Opens a combine archive and extracts sbml models as a list of strings.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for sbml extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == "sbml"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


def extract_rdf_from_combine_archive(archive_path: str):
    """
    Opens a combine archive and extracts annotation string as a list.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for rdf extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == ".rdf"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


#############################################################################################
#   Demonstration of OmexMeta Specification v1.1
#

class OmexMetaSpec1_1:
    """
    This class is a container around some examples of using libOmexMeta via the
    pyomexmeta python front end. Each example is self contained and has corresponding
    method calls in both C++ and C, if either of those are you language of choice.
    """

    def section2_3_4_model_level_annotations(self):
        """Example of how to create model level annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="ModelLevelAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")
        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("ModelLevelAnnotations")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        editor.add_creator("orcid:0000-0001-8254-4957")\
            .add_curator("orcid:0000-0001-8254-4958")\
            .add_taxon("taxon/9895")\
            .add_pubmed("pubmed/12334")\
            .add_description("My supercool model")\
            .add_date_created("2020-09-18")\
            .add_parent_model("pubmed/123456")

        fname = create_combine_archive(sbml, "ModelLevelAnnotations", str(rdf))

        return fname

    def section2_3_4_personal_information(self):
        """Example of how to use the PersonalInformation class"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PersonalInformation")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PersonalInformation")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        # Here we use a "with" block.
        # internally, the "with" block will always execute a piece of code before
        # exiting the with block. In this case, it calls "Editor.add_personal_information()"
        # so that the user cannot forget to do it.
        # Note that in C or C++, the user must remember to add a newly created annotation to the editor.
        with editor.new_personal_information() as information:
            information.add_creator("orcid:0000-0001-8254-4957") \
                .add_name("Robin hood") \
                .add_mbox("rhood@theifinthenight.com") \
                .add_account_name("stolen_goods") \
                .add_account_service_homepage("https://get-your-free-stuff-here.com")

        fname = create_combine_archive(sbml, "PersonalInformation", str(rdf))

        return fname

    def section2_3_6_singular_annotations(self):
        """Example of how to create singular annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="SingularAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("SingularAnnotations")

        # here we turn off semantic extraction to make output clearer
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        with editor.new_singular_annotation() as example_using_bqbiol_pred_and_uri_resource:
            # CHEBI:16236 = ethanol
            example_using_bqbiol_pred_and_uri_resource.about("S") \
                .predicate("bqbiol", "is") \
                .resource_uri("CHEBI:16236")

        with editor.new_singular_annotation() as example_using_bqmodel_pred_and_literal_resource:
            example_using_bqmodel_pred_and_literal_resource.about("MichaelisMenten") \
                .predicate("bqmodel", "isDescribedBy") \
                .resource_literal("This is the Michaelis-Menten schema.")

        # recreate the combine archive
        fname = create_combine_archive(sbml, "SingularAnnotations", str(rdf))

        return fname

    def section2_3_7_1_physical_entity(self):
        """Example of how to create physical entity type composite annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalEntity")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalEntity")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        fname = create_combine_archive(sbml, "PhysicalEntity", str(rdf))

        return fname

    def section2_3_7_2_physical_process(self):
        """Example of how to create PhysicalProcess type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalProcess")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalProcess")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # physical process composite annotations use references to physical entities.
        # therefore we build on the content from OmexMetaSpec1_1.section2_3_7_1_physical_entity()

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        ## optionally print out xml to look at the metaids
        # print(editor.get_xml())
        # print(editor.get_metaids())

        # We now create annotations for three reactions (physical processes)
        #  that are simulated in the sbml model
        with editor.new_physical_process() as substrate_bind_enzyme:
            substrate_bind_enzyme.about("R1", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        with editor.new_physical_process() as substrate_unbind_enzyme:
            substrate_unbind_enzyme.about("R2", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "S") \
                .add_sink(1.0, "E") \
                .add_source(1.0, "ES")

        with editor.new_physical_process() as product_formation:
            product_formation.about("R3", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "E") \
                .add_sink(1.0, "P") \
                .add_source(1.0, "ES")

        fname = create_combine_archive(sbml, "PhysicalProcess", str(rdf))

        return fname

    def section2_3_7_3_energy_diff(self):
        """Example of how to create EnergyDiff type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="EnergyDiff")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("EnergyDiff")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # OPB:OPB_00378 = chemical potential
        with editor.new_energy_diff() as physcial_force:
            physcial_force.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00378") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        fname = create_combine_archive(sbml, "EnergyDiff", str(rdf))

        return fname


if __name__ == "__main__":

    # when True, delete all created omex archives.
    CLEAN_UP = False

    spec_examples = OmexMetaSpec1_1()
    spec_examples.section2_3_4_model_level_annotations()
    spec_examples.section2_3_4_personal_information()
    spec_examples.section2_3_6_singular_annotations()
    spec_examples.section2_3_7_1_physical_entity()
    spec_examples.section2_3_7_2_physical_process()
    spec_examples.section2_3_7_3_energy_diff()

    directory = os.path.join(os.path.dirname(os.path.abspath(__file__)), "*.omex")
    omex_files = glob.glob(directory)

    # error if we have not created 6 omex files.
    # When this script runs without error, we assume we've passed.
    if len(omex_files) != 6:
        raise FileNotFoundError("Something has gone wrong. You should have 6 omex files")

    if CLEAN_UP:
        # prevents cluttering of the repository.
        [os.remove(i) for i in omex_files]

import glob
import os
import libcombine
from pyomexmeta import RDF, eUriType


SBML_STRING = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="MichaelisMenten" id="MichaelisMenten">
    <listOfCompartments>
      <compartment id="Cell" metaid="Cell" spatialDimensions="3" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S" metaid="S" compartment="Cell" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="E" metaid="E" compartment="Cell" initialConcentration="0.5" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="ES" metaid="ES" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="P" metaid="P" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kf" metaid="kf" value="0.1" constant="true"/>
      <parameter id="kb" metaid="kb" value="0.01" constant="true"/>
      <parameter id="kcat" metaid="kcat" value="0.0001" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kf </ci>
              <ci> S </ci>
              <ci> E </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R2" metaid="R2" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kb </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R3" metaid="R3" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
          <speciesReference species="P" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kcat </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


##########################################################################################
#   Helper functions
#   -----------------
#


def create_combine_archive(sbml, name, annotation_string=None):
    """
    Create a combine archive using libcombine. Archive will contain
    a single sbml file with the Manifest that is autocreated by libcombine.
    If annotation_string is not None, an additional annotation file will be
    stored in the combine containing annotation_string.
    Args:
        sbml: the sbml string to be put into a file
        name: Name used for archive and file in archive
        annotation_string: None or string containing the annotation for the combine archive

    Returns:

    """
    current_directory = dir_path = os.path.dirname(os.path.realpath(__file__))
    combine_archive_filename = os.path.join(current_directory, f"{name}.omex")
    archive = libcombine.CombineArchive()
    archive.addFileFromString(
        content=sbml,  # the string contining sbml
        targetName=f"{name}.sbml",  # name the content in archive
        format="sbml",  # format of this content
        isMaster=True  # boolean indicating whether the file should be opened first if there are multiple ones.
    )
    if annotation_string:
        archive.addFileFromString(
            annotation_string,
            f"{name}.rdf",
            "turtle",
            False
        )
    archive.writeToFile(combine_archive_filename)
    if not os.path.isfile(combine_archive_filename):
        raise FileNotFoundError(combine_archive_filename)

    print(f"Archive written to \"{combine_archive_filename}\"")
    return combine_archive_filename


def extract_sbml_from_combine_archive(archive_path):
    """
    Opens a combine archive and extracts sbml models as a list of strings.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for sbml extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == "sbml"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


def extract_rdf_from_combine_archive(archive_path: str):
    """
    Opens a combine archive and extracts annotation string as a list.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for rdf extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == ".rdf"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


#############################################################################################
#   Demonstration of OmexMeta Specification v1.1
#

class OmexMetaSpec1_1:
    """
    This class is a container around some examples of using libOmexMeta via the
    pyomexmeta python front end. Each example is self contained and has corresponding
    method calls in both C++ and C, if either of those are you language of choice.
    """

    def section2_3_4_model_level_annotations(self):
        """Example of how to create model level annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="ModelLevelAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")
        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("ModelLevelAnnotations")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        editor.add_creator("orcid:0000-0001-8254-4957")\
            .add_curator("orcid:0000-0001-8254-4958")\
            .add_taxon("taxon/9895")\
            .add_pubmed("pubmed/12334")\
            .add_description("My supercool model")\
            .add_date_created("2020-09-18")\
            .add_parent_model("pubmed/123456")

        fname = create_combine_archive(sbml, "ModelLevelAnnotations", str(rdf))

        return fname

    def section2_3_4_personal_information(self):
        """Example of how to use the PersonalInformation class"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PersonalInformation")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PersonalInformation")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        # Here we use a "with" block.
        # internally, the "with" block will always execute a piece of code before
        # exiting the with block. In this case, it calls "Editor.add_personal_information()"
        # so that the user cannot forget to do it.
        # Note that in C or C++, the user must remember to add a newly created annotation to the editor.
        with editor.new_personal_information() as information:
            information.add_creator("orcid:0000-0001-8254-4957") \
                .add_name("Robin hood") \
                .add_mbox("rhood@theifinthenight.com") \
                .add_account_name("stolen_goods") \
                .add_account_service_homepage("https://get-your-free-stuff-here.com")

        fname = create_combine_archive(sbml, "PersonalInformation", str(rdf))

        return fname

    def section2_3_6_singular_annotations(self):
        """Example of how to create singular annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="SingularAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("SingularAnnotations")

        # here we turn off semantic extraction to make output clearer
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        with editor.new_singular_annotation() as example_using_bqbiol_pred_and_uri_resource:
            # CHEBI:16236 = ethanol
            example_using_bqbiol_pred_and_uri_resource.about("S") \
                .predicate("bqbiol", "is") \
                .resource_uri("CHEBI:16236")

        with editor.new_singular_annotation() as example_using_bqmodel_pred_and_literal_resource:
            example_using_bqmodel_pred_and_literal_resource.about("MichaelisMenten") \
                .predicate("bqmodel", "isDescribedBy") \
                .resource_literal("This is the Michaelis-Menten schema.")

        # recreate the combine archive
        fname = create_combine_archive(sbml, "SingularAnnotations", str(rdf))

        return fname

    def section2_3_7_1_physical_entity(self):
        """Example of how to create physical entity type composite annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalEntity")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalEntity")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        fname = create_combine_archive(sbml, "PhysicalEntity", str(rdf))

        return fname

    def section2_3_7_2_physical_process(self):
        """Example of how to create PhysicalProcess type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalProcess")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalProcess")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # physical process composite annotations use references to physical entities.
        # therefore we build on the content from OmexMetaSpec1_1.section2_3_7_1_physical_entity()

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        ## optionally print out xml to look at the metaids
        # print(editor.get_xml())
        # print(editor.get_metaids())

        # We now create annotations for three reactions (physical processes)
        #  that are simulated in the sbml model
        with editor.new_physical_process() as substrate_bind_enzyme:
            substrate_bind_enzyme.about("R1", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        with editor.new_physical_process() as substrate_unbind_enzyme:
            substrate_unbind_enzyme.about("R2", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "S") \
                .add_sink(1.0, "E") \
                .add_source(1.0, "ES")

        with editor.new_physical_process() as product_formation:
            product_formation.about("R3", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "E") \
                .add_sink(1.0, "P") \
                .add_source(1.0, "ES")

        fname = create_combine_archive(sbml, "PhysicalProcess", str(rdf))

        return fname

    def section2_3_7_3_energy_diff(self):
        """Example of how to create EnergyDiff type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="EnergyDiff")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("EnergyDiff")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # OPB:OPB_00378 = chemical potential
        with editor.new_energy_diff() as physcial_force:
            physcial_force.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00378") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        fname = create_combine_archive(sbml, "EnergyDiff", str(rdf))

        return fname


if __name__ == "__main__":

    # when True, delete all created omex archives.
    CLEAN_UP = False

    spec_examples = OmexMetaSpec1_1()
    spec_examples.section2_3_4_model_level_annotations()
    spec_examples.section2_3_4_personal_information()
    spec_examples.section2_3_6_singular_annotations()
    spec_examples.section2_3_7_1_physical_entity()
    spec_examples.section2_3_7_2_physical_process()
    spec_examples.section2_3_7_3_energy_diff()

    directory = os.path.join(os.path.dirname(os.path.abspath(__file__)), "*.omex")
    omex_files = glob.glob(directory)

    # error if we have not created 6 omex files.
    # When this script runs without error, we assume we've passed.
    if len(omex_files) != 6:
        raise FileNotFoundError("Something has gone wrong. You should have 6 omex files")

    if CLEAN_UP:
        # prevents cluttering of the repository.
        [os.remove(i) for i in omex_files]

In [ ]:
UPDATE ME

from pyomexmeta import RDF, eUriType
rdf = RDF()
rdf.set_archive_uri("GlucoseTransport.omex")
rdf.set_model_uri("glucose_transport.sbml")
# resuing the sbml variable from the previous section
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = The GO cellular component term for cytoplasm.
    # OPB:00340 = chemical concentration
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                     .identity("CHEBI:17234")\
                     .is_part_of("GO:0005737")\
                     .has_property("OPB:00340")

print(rdf)

Note that the difference between this and the last example is the presence of the additional annotation
with the `bqbiol:isVersionOf` predicate:

    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .


<left> <h5> 3.2 Annotations for a Physical Process </h5> </left>
Perhaps the most prevalent type of biosimulation model is one that simulates biochemical reactions.
Thus, we provide special attention to annotations for biological processes, such as reactions.
Unlike physical entities, processes rarely have unique names. However, some processes can
be categorized as a “version of” a Gene Ontology process term. Thus, in the main text, we used
a “version of” alcohol dehydrogenase. Below, we provide the Python code for an expanded version
of this same reaction. Note, we reuse the sbml model from an earlier example but include the sbml
string to make the example self contained.

In [ ]:
from pyomexmeta import RDF, eUriType

cellml = """"""""

UPDATE ME

rdf = RDF()
rdf.set_archive_uri("AHD.omex")
rdf.set_model_uri("AHDModel.sbml")
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_process() as phy_process:
    phy_process \
        .about("ADH") \
        .is_version_of("GO:0004022") \
        .add_source(physical_entity_reference="EtOH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_source(physical_entity_reference="NAD", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="Aldehyde", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="NADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_mediator(physical_entity_reference="ADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .has_property("OPB:OPB_00237")                       #Chemical flow rate

print(rdf)

UPDATE ME

In the above, the reaction has two reactants, two products, and one mediating enzyme.
Note that stoichiometry, where appropriate, can be specified via the multiplier fields.
Like with the physical entity type composite annotations, the `about` method
provides the link between the annotation document we are composing and the SBML source code. Similarly,
the `physical_entity_reference` arguments are metaids to elements within the SBML document.

The final optional clause `has_property` indicates that the reaction being simulated has the physical property
of “chemical flow rate”.

<left> <h5> 3.3 Annotations for a energy differential </h5> </left>

*** attention This example needs work. Please provide a model and fix the annotation apropriately ***

Finally, we provide an example for annotating the electrical potential across the cellular membrane,
specifically resulting from calcium ions. In addition to source and sink, this is annotated with the
OPB term for the Nernst, or reversal potential.

The next example demonstrates the annotation of a physical entity in a model that simulates
hemodynamics. We annotate a term as capturing the blood pressure (physical property) within the aorta (physical entity).

In [ ]:
from pyomexmeta import RDF, eUriType
crllml = we need a model for this example.

rdf = RDF()
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_entity() as blood_pressure_entity:
    # opb/OPB:00509 = fluid pressure
    # fma/fma_9670 = portion of blood
    # #lumen of Aorta = lumen of aorta
    blood_pressure_entity.about("Pa001") \
                        .has_property("opb/OPB:00509") \
                        .identity("fma/fma_9670") \
                        .is_part_of("fma/fma_13096")



In [ ]:
rdf = RDF()
rdf.set_archive_uri("AHD.omex")
rdf.set_model_uri("AHDModel.sbml")

# Omex_S001: Calcium Ions extracellular space
# EnergyD001: The metaID of the SBML parameter
# Omex_S003: Calcium ions in cytoplasm
#OPB/OPB_01581: Nernst reversal potential
with annot_editor.new_physical_energyDiff() as energy_diff:
    energy_diff \
     .about("EnergyD001") \
     .add_source(physical_entity_reference="Omex_S001", uri_type=eUriType.MODEL_URI) \
     .add_sink(physical_entity_reference="Omex_S003", uri_type=eUriType.MODEL_URI) \
     .add_has_property("OPB/OPB_01581")


<center> <h2> Part B: libOmexMeta Examples </h2> </center>
While the preceding section is designed to be more tutorial based to
guide users around the libOmexMeta library, this section is designed to be
example led, with an emphasis on how one is likely to use libOmexMeta in conjuction with `libcombine`.

In these examples, we use a simple SBML model that simulates the traditional Michaelis-Menten schema.     

In [ ]:
import glob
import os
import libcombine
from pyomexmeta import RDF, eUriType


SBML_STRING = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="MichaelisMenten" id="MichaelisMenten">
    <listOfCompartments>
      <compartment id="Cell" metaid="Cell" spatialDimensions="3" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S" metaid="S" compartment="Cell" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="E" metaid="E" compartment="Cell" initialConcentration="0.5" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="ES" metaid="ES" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="P" metaid="P" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kf" metaid="kf" value="0.1" constant="true"/>
      <parameter id="kb" metaid="kb" value="0.01" constant="true"/>
      <parameter id="kcat" metaid="kcat" value="0.0001" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kf </ci>
              <ci> S </ci>
              <ci> E </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R2" metaid="R2" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kb </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R3" metaid="R3" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
          <speciesReference species="P" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kcat </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


##########################################################################################
#   Helper functions
#   -----------------
#


def create_combine_archive(sbml, name, annotation_string=None):
    """
    Create a combine archive using libcombine. Archive will contain
    a single sbml file with the Manifest that is autocreated by libcombine.
    If annotation_string is not None, an additional annotation file will be
    stored in the combine containing annotation_string.
    Args:
        sbml: the sbml string to be put into a file
        name: Name used for archive and file in archive
        annotation_string: None or string containing the annotation for the combine archive

    Returns:

    """
    current_directory = dir_path = os.path.dirname(os.path.realpath(__file__))
    combine_archive_filename = os.path.join(current_directory, f"{name}.omex")
    archive = libcombine.CombineArchive()
    archive.addFileFromString(
        content=sbml,  # the string contining sbml
        targetName=f"{name}.sbml",  # name the content in archive
        format="sbml",  # format of this content
        isMaster=True  # boolean indicating whether the file should be opened first if there are multiple ones.
    )
    if annotation_string:
        archive.addFileFromString(
            annotation_string,
            f"{name}.rdf",
            "turtle",
            False
        )
    archive.writeToFile(combine_archive_filename)
    if not os.path.isfile(combine_archive_filename):
        raise FileNotFoundError(combine_archive_filename)

    print(f"Archive written to \"{combine_archive_filename}\"")
    return combine_archive_filename


def extract_sbml_from_combine_archive(archive_path):
    """
    Opens a combine archive and extracts sbml models as a list of strings.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for sbml extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == "sbml"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


def extract_rdf_from_combine_archive(archive_path: str):
    """
    Opens a combine archive and extracts annotation string as a list.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for rdf extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == ".rdf"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


#############################################################################################
#   Demonstration of OmexMeta Specification v1.1
#

class OmexMetaSpec1_1:
    """
    This class is a container around some examples of using libOmexMeta via the
    pyomexmeta python front end. Each example is self contained and has corresponding
    method calls in both C++ and C, if either of those are you language of choice.
    """

    def section2_3_4_model_level_annotations(self):
        """Example of how to create model level annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="ModelLevelAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")
        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("ModelLevelAnnotations")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        editor.add_creator("orcid:0000-0001-8254-4957")\
            .add_curator("orcid:0000-0001-8254-4958")\
            .add_taxon("taxon/9895")\
            .add_pubmed("pubmed/12334")\
            .add_description("My supercool model")\
            .add_date_created("2020-09-18")\
            .add_parent_model("pubmed/123456")

        fname = create_combine_archive(sbml, "ModelLevelAnnotations", str(rdf))

        return fname

    def section2_3_4_personal_information(self):
        """Example of how to use the PersonalInformation class"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PersonalInformation")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PersonalInformation")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        # Here we use a "with" block.
        # internally, the "with" block will always execute a piece of code before
        # exiting the with block. In this case, it calls "Editor.add_personal_information()"
        # so that the user cannot forget to do it.
        # Note that in C or C++, the user must remember to add a newly created annotation to the editor.
        with editor.new_personal_information() as information:
            information.add_creator("orcid:0000-0001-8254-4957") \
                .add_name("Robin hood") \
                .add_mbox("rhood@theifinthenight.com") \
                .add_account_name("stolen_goods") \
                .add_account_service_homepage("https://get-your-free-stuff-here.com")

        fname = create_combine_archive(sbml, "PersonalInformation", str(rdf))

        return fname

    def section2_3_6_singular_annotations(self):
        """Example of how to create singular annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="SingularAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("SingularAnnotations")

        # here we turn off semantic extraction to make output clearer
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        with editor.new_singular_annotation() as example_using_bqbiol_pred_and_uri_resource:
            # CHEBI:16236 = ethanol
            example_using_bqbiol_pred_and_uri_resource.about("S") \
                .predicate("bqbiol", "is") \
                .resource_uri("CHEBI:16236")

        with editor.new_singular_annotation() as example_using_bqmodel_pred_and_literal_resource:
            example_using_bqmodel_pred_and_literal_resource.about("MichaelisMenten") \
                .predicate("bqmodel", "isDescribedBy") \
                .resource_literal("This is the Michaelis-Menten schema.")

        # recreate the combine archive
        fname = create_combine_archive(sbml, "SingularAnnotations", str(rdf))

        return fname

    def section2_3_7_1_physical_entity(self):
        """Example of how to create physical entity type composite annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalEntity")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalEntity")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        fname = create_combine_archive(sbml, "PhysicalEntity", str(rdf))

        return fname

    def section2_3_7_2_physical_process(self):
        """Example of how to create PhysicalProcess type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalProcess")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalProcess")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # physical process composite annotations use references to physical entities.
        # therefore we build on the content from OmexMetaSpec1_1.section2_3_7_1_physical_entity()

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        ## optionally print out xml to look at the metaids
        # print(editor.get_xml())
        # print(editor.get_metaids())

        # We now create annotations for three reactions (physical processes)
        #  that are simulated in the sbml model
        with editor.new_physical_process() as substrate_bind_enzyme:
            substrate_bind_enzyme.about("R1", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        with editor.new_physical_process() as substrate_unbind_enzyme:
            substrate_unbind_enzyme.about("R2", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "S") \
                .add_sink(1.0, "E") \
                .add_source(1.0, "ES")

        with editor.new_physical_process() as product_formation:
            product_formation.about("R3", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "E") \
                .add_sink(1.0, "P") \
                .add_source(1.0, "ES")

        fname = create_combine_archive(sbml, "PhysicalProcess", str(rdf))

        return fname

    def section2_3_7_3_energy_diff(self):
        """Example of how to create EnergyDiff type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="EnergyDiff")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("EnergyDiff")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # OPB:OPB_00378 = chemical potential
        with editor.new_energy_diff() as physcial_force:
            physcial_force.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00378") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        fname = create_combine_archive(sbml, "EnergyDiff", str(rdf))

        return fname


if __name__ == "__main__":

    # when True, delete all created omex archives.
    CLEAN_UP = False

    spec_examples = OmexMetaSpec1_1()
    spec_examples.section2_3_4_model_level_annotations()
    spec_examples.section2_3_4_personal_information()
    spec_examples.section2_3_6_singular_annotations()
    spec_examples.section2_3_7_1_physical_entity()
    spec_examples.section2_3_7_2_physical_process()
    spec_examples.section2_3_7_3_energy_diff()

    directory = os.path.join(os.path.dirname(os.path.abspath(__file__)), "*.omex")
    omex_files = glob.glob(directory)

    # error if we have not created 6 omex files.
    # When this script runs without error, we assume we've passed.
    if len(omex_files) != 6:
        raise FileNotFoundError("Something has gone wrong. You should have 6 omex files")

    if CLEAN_UP:
        # prevents cluttering of the repository.
        [os.remove(i) for i in omex_files]

import glob
import os
import libcombine
from pyomexmeta import RDF, eUriType


SBML_STRING = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="MichaelisMenten" id="MichaelisMenten">
    <listOfCompartments>
      <compartment id="Cell" metaid="Cell" spatialDimensions="3" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S" metaid="S" compartment="Cell" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="E" metaid="E" compartment="Cell" initialConcentration="0.5" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="ES" metaid="ES" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="P" metaid="P" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kf" metaid="kf" value="0.1" constant="true"/>
      <parameter id="kb" metaid="kb" value="0.01" constant="true"/>
      <parameter id="kcat" metaid="kcat" value="0.0001" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kf </ci>
              <ci> S </ci>
              <ci> E </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R2" metaid="R2" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kb </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R3" metaid="R3" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
          <speciesReference species="P" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kcat </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


##########################################################################################
#   Helper functions
#   -----------------
#


def create_combine_archive(sbml, name, annotation_string=None):
    """
    Create a combine archive using libcombine. Archive will contain
    a single sbml file with the Manifest that is autocreated by libcombine.
    If annotation_string is not None, an additional annotation file will be
    stored in the combine containing annotation_string.
    Args:
        sbml: the sbml string to be put into a file
        name: Name used for archive and file in archive
        annotation_string: None or string containing the annotation for the combine archive

    Returns:

    """
    current_directory = dir_path = os.path.dirname(os.path.realpath(__file__))
    combine_archive_filename = os.path.join(current_directory, f"{name}.omex")
    archive = libcombine.CombineArchive()
    archive.addFileFromString(
        content=sbml,  # the string contining sbml
        targetName=f"{name}.sbml",  # name the content in archive
        format="sbml",  # format of this content
        isMaster=True  # boolean indicating whether the file should be opened first if there are multiple ones.
    )
    if annotation_string:
        archive.addFileFromString(
            annotation_string,
            f"{name}.rdf",
            "turtle",
            False
        )
    archive.writeToFile(combine_archive_filename)
    if not os.path.isfile(combine_archive_filename):
        raise FileNotFoundError(combine_archive_filename)

    print(f"Archive written to \"{combine_archive_filename}\"")
    return combine_archive_filename


def extract_sbml_from_combine_archive(archive_path):
    """
    Opens a combine archive and extracts sbml models as a list of strings.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for sbml extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == "sbml"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


def extract_rdf_from_combine_archive(archive_path: str):
    """
    Opens a combine archive and extracts annotation string as a list.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for rdf extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == ".rdf"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


#############################################################################################
#   Demonstration of OmexMeta Specification v1.1
#

class OmexMetaSpec1_1:
    """
    This class is a container around some examples of using libOmexMeta via the
    pyomexmeta python front end. Each example is self contained and has corresponding
    method calls in both C++ and C, if either of those are you language of choice.
    """

    def section2_3_4_model_level_annotations(self):
        """Example of how to create model level annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="ModelLevelAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")
        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("ModelLevelAnnotations")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        editor.add_creator("orcid:0000-0001-8254-4957")\
            .add_curator("orcid:0000-0001-8254-4958")\
            .add_taxon("taxon/9895")\
            .add_pubmed("pubmed/12334")\
            .add_description("My supercool model")\
            .add_date_created("2020-09-18")\
            .add_parent_model("pubmed/123456")

        fname = create_combine_archive(sbml, "ModelLevelAnnotations", str(rdf))

        return fname

    def section2_3_4_personal_information(self):
        """Example of how to use the PersonalInformation class"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PersonalInformation")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PersonalInformation")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        # Here we use a "with" block.
        # internally, the "with" block will always execute a piece of code before
        # exiting the with block. In this case, it calls "Editor.add_personal_information()"
        # so that the user cannot forget to do it.
        # Note that in C or C++, the user must remember to add a newly created annotation to the editor.
        with editor.new_personal_information() as information:
            information.add_creator("orcid:0000-0001-8254-4957") \
                .add_name("Robin hood") \
                .add_mbox("rhood@theifinthenight.com") \
                .add_account_name("stolen_goods") \
                .add_account_service_homepage("https://get-your-free-stuff-here.com")

        fname = create_combine_archive(sbml, "PersonalInformation", str(rdf))

        return fname

    def section2_3_6_singular_annotations(self):
        """Example of how to create singular annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="SingularAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("SingularAnnotations")

        # here we turn off semantic extraction to make output clearer
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        with editor.new_singular_annotation() as example_using_bqbiol_pred_and_uri_resource:
            # CHEBI:16236 = ethanol
            example_using_bqbiol_pred_and_uri_resource.about("S") \
                .predicate("bqbiol", "is") \
                .resource_uri("CHEBI:16236")

        with editor.new_singular_annotation() as example_using_bqmodel_pred_and_literal_resource:
            example_using_bqmodel_pred_and_literal_resource.about("MichaelisMenten") \
                .predicate("bqmodel", "isDescribedBy") \
                .resource_literal("This is the Michaelis-Menten schema.")

        # recreate the combine archive
        fname = create_combine_archive(sbml, "SingularAnnotations", str(rdf))

        return fname

    def section2_3_7_1_physical_entity(self):
        """Example of how to create physical entity type composite annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalEntity")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalEntity")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        fname = create_combine_archive(sbml, "PhysicalEntity", str(rdf))

        return fname

    def section2_3_7_2_physical_process(self):
        """Example of how to create PhysicalProcess type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalProcess")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalProcess")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # physical process composite annotations use references to physical entities.
        # therefore we build on the content from OmexMetaSpec1_1.section2_3_7_1_physical_entity()

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        ## optionally print out xml to look at the metaids
        # print(editor.get_xml())
        # print(editor.get_metaids())

        # We now create annotations for three reactions (physical processes)
        #  that are simulated in the sbml model
        with editor.new_physical_process() as substrate_bind_enzyme:
            substrate_bind_enzyme.about("R1", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        with editor.new_physical_process() as substrate_unbind_enzyme:
            substrate_unbind_enzyme.about("R2", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "S") \
                .add_sink(1.0, "E") \
                .add_source(1.0, "ES")

        with editor.new_physical_process() as product_formation:
            product_formation.about("R3", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "E") \
                .add_sink(1.0, "P") \
                .add_source(1.0, "ES")

        fname = create_combine_archive(sbml, "PhysicalProcess", str(rdf))

        return fname

    def section2_3_7_3_energy_diff(self):
        """Example of how to create EnergyDiff type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="EnergyDiff")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("EnergyDiff")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # OPB:OPB_00378 = chemical potential
        with editor.new_energy_diff() as physcial_force:
            physcial_force.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00378") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        fname = create_combine_archive(sbml, "EnergyDiff", str(rdf))

        return fname


if __name__ == "__main__":

    # when True, delete all created omex archives.
    CLEAN_UP = False

    spec_examples = OmexMetaSpec1_1()
    spec_examples.section2_3_4_model_level_annotations()
    spec_examples.section2_3_4_personal_information()
    spec_examples.section2_3_6_singular_annotations()
    spec_examples.section2_3_7_1_physical_entity()
    spec_examples.section2_3_7_2_physical_process()
    spec_examples.section2_3_7_3_energy_diff()

    directory = os.path.join(os.path.dirname(os.path.abspath(__file__)), "*.omex")
    omex_files = glob.glob(directory)

    # error if we have not created 6 omex files.
    # When this script runs without error, we assume we've passed.
    if len(omex_files) != 6:
        raise FileNotFoundError("Something has gone wrong. You should have 6 omex files")

    if CLEAN_UP:
        # prevents cluttering of the repository.
        [os.remove(i) for i in omex_files]

In [5]:
UPDATE ME

from pyomexmeta import RDF, eUriType
rdf = RDF()
rdf.set_archive_uri("GlucoseTransport.omex")
rdf.set_model_uri("glucose_transport.sbml")
# resuing the sbml variable from the previous section
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)
with annot_editor.new_physical_entity() as cytosolic_glucose:
    # CHEBI:17234 = glucose
    # GO:0005737 = The GO cellular component term for cytoplasm.
    # OPB:00340 = chemical concentration
    cytosolic_glucose.about("glucose_c", eUriType.MODEL_URI) \
                     .identity("CHEBI:17234")\
                     .is_part_of("GO:0005737")\
                     .has_property("OPB:00340")

print(rdf)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix bqbiol: <http://biomodels.net/biology-qualifiers/> .
@prefix OMEXlib: <http://omex-library.org/> .
@prefix myOMEX: <http://omex-library.org/GlucoseTransport.omex/> .
@prefix local: <http://omex-library.org/GlucoseTransport.omex/glucose_transport.rdf#> .

local:EntityProperty0000
    bqbiol:isPropertyOf <http://omex-library.org/GlucoseTransport.omex/glucose_transport.sbml#glucose_c> ;
    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .

<http://omex-library.org/GlucoseTransport.omex/glucose_transport.sbml#glucose_c>
    bqbiol:is <https://identifiers.org/CHEBI:17234> ;
    bqbiol:isPartOf <https://identifiers.org/GO:0005737> .




Note that the difference between this and the last example is the presence of the additional annotation
with the `bqbiol:isVersionOf` predicate:

    bqbiol:isVersionOf <https://identifiers.org/OPB:00340> .


<left> <h5> 3.2 Annotations for a Physical Process </h5> </left>
Perhaps the most prevalent type of biosimulation model is one that simulates biochemical reactions.
Thus, we provide special attention to annotations for biological processes, such as reactions.
Unlike physical entities, processes rarely have unique names. However, some processes can
be categorized as a “version of” a Gene Ontology process term. Thus, in the main text, we used
a “version of” alcohol dehydrogenase. Below, we provide the Python code for an expanded version
of this same reaction. Note, we reuse the sbml model from an earlier example but include the sbml
string to make the example self contained.

In [6]:
from pyomexmeta import RDF, eUriType

cellml = """"""""

UPDATE ME

rdf = RDF()
rdf.set_archive_uri("AHD.omex")
rdf.set_model_uri("AHDModel.sbml")
annot_editor = rdf.to_editor(cellml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_process() as phy_process:
    phy_process \
        .about("ADH") \
        .is_version_of("GO:0004022") \
        .add_source(physical_entity_reference="EtOH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_source(physical_entity_reference="NAD", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="Aldehyde", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_sink(physical_entity_reference="NADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .add_mediator(physical_entity_reference="ADH", uri_type=eUriType.MODEL_URI, multiplier=1.0) \
        .has_property("OPB:OPB_00237")                       #Chemical flow rate

print(rdf)

TypeError: about() missing 1 required positional argument: 'uri_type'

UPDATE ME

In the above, the reaction has two reactants, two products, and one mediating enzyme.
Note that stoichiometry, where appropriate, can be specified via the multiplier fields.
Like with the physical entity type composite annotations, the `about` method
provides the link between the annotation document we are composing and the SBML source code. Similarly,
the `physical_entity_reference` arguments are metaids to elements within the SBML document.

The final optional clause `has_property` indicates that the reaction being simulated has the physical property
of “chemical flow rate”.

<left> <h5> 3.3 Annotations for a energy differential </h5> </left>

*** attention This example needs work. Please provide a model and fix the annotation apropriately ***

Finally, we provide an example for annotating the electrical potential across the cellular membrane,
specifically resulting from calcium ions. In addition to source and sink, this is annotated with the
OPB term for the Nernst, or reversal potential.

The next example demonstrates the annotation of a physical entity in a model that simulates
hemodynamics. We annotate a term as capturing the blood pressure (physical property) within the aorta (physical entity).

In [ ]:
from pyomexmeta import RDF, eUriType
crllml = we need a model for this example.

rdf = RDF()
annot_editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

with annot_editor.new_physical_entity() as blood_pressure_entity:
    # opb/OPB:00509 = fluid pressure
    # fma/fma_9670 = portion of blood
    # #lumen of Aorta = lumen of aorta
    blood_pressure_entity.about("Pa001") \
                        .has_property("opb/OPB:00509") \
                        .identity("fma/fma_9670") \
                        .is_part_of("fma/fma_13096")



In [ ]:
rdf = RDF()
rdf.set_archive_uri("AHD.omex")
rdf.set_model_uri("AHDModel.sbml")

# Omex_S001: Calcium Ions extracellular space
# EnergyD001: The metaID of the SBML parameter
# Omex_S003: Calcium ions in cytoplasm
#OPB/OPB_01581: Nernst reversal potential
with annot_editor.new_physical_energyDiff() as energy_diff:
    energy_diff \
     .about("EnergyD001") \
     .add_source(physical_entity_reference="Omex_S001", uri_type=eUriType.MODEL_URI) \
     .add_sink(physical_entity_reference="Omex_S003", uri_type=eUriType.MODEL_URI) \
     .add_has_property("OPB/OPB_01581")


<center> <h2> Part B: libOmexMeta Examples </h2> </center>
While the preceding section is designed to be more tutorial based to
guide users around the libOmexMeta library, this section is designed to be
example led, with an emphasis on how one is likely to use libOmexMeta in conjuction with `libcombine`.

In these examples, we use a simple SBML model that simulates the traditional Michaelis-Menten schema.     

In [ ]:
import glob
import os
import libcombine
from pyomexmeta import RDF, eUriType


SBML_STRING = """<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" level="3" version="1">
  <model metaid="MichaelisMenten" id="MichaelisMenten">
    <listOfCompartments>
      <compartment id="Cell" metaid="Cell" spatialDimensions="3" constant="true"/>
    </listOfCompartments>
    <listOfSpecies>
      <species id="S" metaid="S" compartment="Cell" initialConcentration="100" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="E" metaid="E" compartment="Cell" initialConcentration="0.5" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="ES" metaid="ES" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
      <species id="P" metaid="P" compartment="Cell" initialConcentration="0" hasOnlySubstanceUnits="false" boundaryCondition="false" constant="false"/>
    </listOfSpecies>
    <listOfParameters>
      <parameter id="kf" metaid="kf" value="0.1" constant="true"/>
      <parameter id="kb" metaid="kb" value="0.01" constant="true"/>
      <parameter id="kcat" metaid="kcat" value="0.0001" constant="true"/>
    </listOfParameters>
    <listOfReactions>
      <reaction id="R1" metaid="R1" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kf </ci>
              <ci> S </ci>
              <ci> E </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R2" metaid="R2" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="S" stoichiometry="1" constant="true"/>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kb </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
      <reaction id="R3" metaid="R3" reversible="false" fast="false">
        <listOfReactants>
          <speciesReference species="ES" stoichiometry="1" constant="true"/>
        </listOfReactants>
        <listOfProducts>
          <speciesReference species="E" stoichiometry="1" constant="true"/>
          <speciesReference species="P" stoichiometry="1" constant="true"/>
        </listOfProducts>
        <kineticLaw>
          <math xmlns="http://www.w3.org/1998/Math/MathML">
            <apply>
              <times/>
              <ci> kcat </ci>
              <ci> ES </ci>
            </apply>
          </math>
        </kineticLaw>
      </reaction>
    </listOfReactions>
  </model>
</sbml>
"""


##########################################################################################
#   Helper functions
#   -----------------
#


def create_combine_archive(sbml, name, annotation_string=None):
    """
    Create a combine archive using libcombine. Archive will contain
    a single sbml file with the Manifest that is autocreated by libcombine.
    If annotation_string is not None, an additional annotation file will be
    stored in the combine containing annotation_string.
    Args:
        sbml: the sbml string to be put into a file
        name: Name used for archive and file in archive
        annotation_string: None or string containing the annotation for the combine archive

    Returns:

    """
    current_directory = dir_path = os.path.dirname(os.path.realpath(__file__))
    combine_archive_filename = os.path.join(current_directory, f"{name}.omex")
    archive = libcombine.CombineArchive()
    archive.addFileFromString(
        content=sbml,  # the string contining sbml
        targetName=f"{name}.sbml",  # name the content in archive
        format="sbml",  # format of this content
        isMaster=True  # boolean indicating whether the file should be opened first if there are multiple ones.
    )
    if annotation_string:
        archive.addFileFromString(
            annotation_string,
            f"{name}.rdf",
            "turtle",
            False
        )
    archive.writeToFile(combine_archive_filename)
    if not os.path.isfile(combine_archive_filename):
        raise FileNotFoundError(combine_archive_filename)

    print(f"Archive written to \"{combine_archive_filename}\"")
    return combine_archive_filename


def extract_sbml_from_combine_archive(archive_path):
    """
    Opens a combine archive and extracts sbml models as a list of strings.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for sbml extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == "sbml"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


def extract_rdf_from_combine_archive(archive_path: str):
    """
    Opens a combine archive and extracts annotation string as a list.
    Args:
        archive_path: full path to combine archive on disk

    Returns:

    """
    if not os.path.isfile(archive_path):
        raise FileNotFoundError(archive_path)

    # read the archive using libcombine
    archive = libcombine.CombineArchive()

    # note the skipOmex flag. This is needed to expose any files with an "rdf" extension.
    archive.initializeFromArchive(archive_path, skipOmex=True)  # new in libcombine!

    # filter through the entries in the omex archive for rdf extension files
    annotation_entries = [i.c_str() for i in archive.getAllLocations() if i[-4:] == ".rdf"]

    # read the rdf into a python string
    return [archive.extractEntryToString(i) for i in annotation_entries]


#############################################################################################
#   Demonstration of OmexMeta Specification v1.1
#

class OmexMetaSpec1_1:
    """
    This class is a container around some examples of using libOmexMeta via the
    pyomexmeta python front end. Each example is self contained and has corresponding
    method calls in both C++ and C, if either of those are you language of choice.
    """

    def section2_3_4_model_level_annotations(self):
        """Example of how to create model level annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="ModelLevelAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")
        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("ModelLevelAnnotations")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        editor.add_creator("orcid:0000-0001-8254-4957")\
            .add_curator("orcid:0000-0001-8254-4958")\
            .add_taxon("taxon/9895")\
            .add_pubmed("pubmed/12334")\
            .add_description("My supercool model")\
            .add_date_created("2020-09-18")\
            .add_parent_model("pubmed/123456")

        fname = create_combine_archive(sbml, "ModelLevelAnnotations", str(rdf))

        return fname

    def section2_3_4_personal_information(self):
        """Example of how to use the PersonalInformation class"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PersonalInformation")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PersonalInformation")

        # create an editor. Note these are the default arguments - but shown here for completeness
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=True)

        # Here we use a "with" block.
        # internally, the "with" block will always execute a piece of code before
        # exiting the with block. In this case, it calls "Editor.add_personal_information()"
        # so that the user cannot forget to do it.
        # Note that in C or C++, the user must remember to add a newly created annotation to the editor.
        with editor.new_personal_information() as information:
            information.add_creator("orcid:0000-0001-8254-4957") \
                .add_name("Robin hood") \
                .add_mbox("rhood@theifinthenight.com") \
                .add_account_name("stolen_goods") \
                .add_account_service_homepage("https://get-your-free-stuff-here.com")

        fname = create_combine_archive(sbml, "PersonalInformation", str(rdf))

        return fname

    def section2_3_6_singular_annotations(self):
        """Example of how to create singular annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="SingularAnnotations")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("SingularAnnotations")

        # here we turn off semantic extraction to make output clearer
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        with editor.new_singular_annotation() as example_using_bqbiol_pred_and_uri_resource:
            # CHEBI:16236 = ethanol
            example_using_bqbiol_pred_and_uri_resource.about("S") \
                .predicate("bqbiol", "is") \
                .resource_uri("CHEBI:16236")

        with editor.new_singular_annotation() as example_using_bqmodel_pred_and_literal_resource:
            example_using_bqmodel_pred_and_literal_resource.about("MichaelisMenten") \
                .predicate("bqmodel", "isDescribedBy") \
                .resource_literal("This is the Michaelis-Menten schema.")

        # recreate the combine archive
        fname = create_combine_archive(sbml, "SingularAnnotations", str(rdf))

        return fname

    def section2_3_7_1_physical_entity(self):
        """Example of how to create physical entity type composite annotations"""
        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalEntity")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalEntity")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        fname = create_combine_archive(sbml, "PhysicalEntity", str(rdf))

        return fname

    def section2_3_7_2_physical_process(self):
        """Example of how to create PhysicalProcess type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="PhysicalProcess")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("PhysicalProcess")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # physical process composite annotations use references to physical entities.
        # therefore we build on the content from OmexMetaSpec1_1.section2_3_7_1_physical_entity()

        # note: the syntax "fma:fma_12345" == "fma/fma_12345"
        # OPB:OPB_00340 = concentration of chemical
        # CHEBI:16236 = ethanol
        # CHEBI:17478 = an aldehyde group
        # GO:0004022 = aldehyde dehydrogenase
        # FMA:66836 = part of cytosol
        # FMA:63877 = fibroblast
        with editor.new_physical_entity() as substrate_entity:
            substrate_entity.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:16236") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as product_entity:
            product_entity.about("P", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("CHEBI:17478") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as enzyme_entity:
            enzyme_entity.about("E", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .identity("GO:00040223") \
                .is_part_of("FMA:66836") \
                .is_part_of("FMA:63877")

        with editor.new_physical_entity() as complex_entity:
            complex_entity.about("ES", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00340") \
                .has_part("GO:0004022") \
                .has_part("CHEBI:16236")

        ## optionally print out xml to look at the metaids
        # print(editor.get_xml())
        # print(editor.get_metaids())

        # We now create annotations for three reactions (physical processes)
        #  that are simulated in the sbml model
        with editor.new_physical_process() as substrate_bind_enzyme:
            substrate_bind_enzyme.about("R1", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        with editor.new_physical_process() as substrate_unbind_enzyme:
            substrate_unbind_enzyme.about("R2", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "S") \
                .add_sink(1.0, "E") \
                .add_source(1.0, "ES")

        with editor.new_physical_process() as product_formation:
            product_formation.about("R3", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00593") \
                .add_sink(1.0, "E") \
                .add_sink(1.0, "P") \
                .add_source(1.0, "ES")

        fname = create_combine_archive(sbml, "PhysicalProcess", str(rdf))

        return fname

    def section2_3_7_3_energy_diff(self):
        """Example of how to create EnergyDiff type composite annotations"""

        # Create a combine archive called MichaelisMenten1.omex that contains a MichaelisMenten1.sbml file
        combine_archive_filename = create_combine_archive(SBML_STRING, name="EnergyDiff")

        # we extract the sbml from the combine archive
        sbml = extract_sbml_from_combine_archive(combine_archive_filename)

        # sbml is a list of all sbml files in archive. We know there is only 1, so get the string
        if len(sbml) != 1:
            raise ValueError("Something is wrong - you should only have 1 sbml file in the combine archive")

        sbml = sbml[0]

        # create an RDF object. Its empty.
        rdf = RDF()
        rdf.set_model_uri("EnergyDiff")

        # sbml semantic extraction will automatically create these physical entities for us.
        # here, we turn it off so we can create them manually
        editor = rdf.to_editor(sbml, generate_new_metaids=False, sbml_semantic_extraction=False)

        # OPB:OPB_00378 = chemical potential
        with editor.new_energy_diff() as physcial_force:
            physcial_force.about("S", eUriType.MODEL_URI) \
                .has_property("OPB:OPB_00378") \
                .add_source(1.0, "S") \
                .add_source(1.0, "E") \
                .add_sink(1.0, "ES")

        fname = create_combine_archive(sbml, "EnergyDiff", str(rdf))

        return fname


if __name__ == "__main__":

    # when True, delete all created omex archives.
    CLEAN_UP = False

    spec_examples = OmexMetaSpec1_1()
    spec_examples.section2_3_4_model_level_annotations()
    spec_examples.section2_3_4_personal_information()
    spec_examples.section2_3_6_singular_annotations()
    spec_examples.section2_3_7_1_physical_entity()
    spec_examples.section2_3_7_2_physical_process()
    spec_examples.section2_3_7_3_energy_diff()

    directory = os.path.join(os.path.dirname(os.path.abspath(__file__)), "*.omex")
    omex_files = glob.glob(directory)

    # error if we have not created 6 omex files.
    # When this script runs without error, we assume we've passed.
    if len(omex_files) != 6:
        raise FileNotFoundError("Something has gone wrong. You should have 6 omex files")

    if CLEAN_UP:
        # prevents cluttering of the repository.
        [os.remove(i) for i in omex_files]